In [1]:
# ==========================================
# CELL 1: DEEP FLIGHT RECORDER (FULL FIDELITY)
# ==========================================
import os
import json
import time
import uuid
import logging
import copy
from datetime import datetime
from typing import TypedDict, List, Dict, Literal, Any
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END

# --- IMPORT CONFIG & MODULES ---
from config import (
    GRAPH_PATH, CHUNKS_PATH, VECTOR_INDEX_PATH, BM25_INDEX_PATH, 
    QUERY_MODEL, SCOUT_MODEL, AUDIT_MODEL, SYNTHESIZE_MODEL,
    GROQ_API_KEY, LOG_FILE_PATH, REPORTS_DIR
)
from rag_engine import OmniRetriever
from web_engine import DeepWebScout, KnowledgeCurator

# --- SETUP LOGGING ---
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(
    filename=LOG_FILE_PATH,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger("BRAIN")

class DeepFlightRecorder:
    def __init__(self):
        self.current_run_id = None
        self.run_data = {}
        if not os.path.exists(REPORTS_DIR): os.makedirs(REPORTS_DIR)

    def start_run(self, query):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.current_run_id = f"trace_{timestamp}"
        self.run_data = {
            "meta": { "run_id": self.current_run_id, "query": query, "timestamp": timestamp },
            "trace_log": []
        }
        print(f"📼 Deep Trace started: {self.current_run_id}")

    def log_event(self, event_type: str, component: str, data: Any, duration_ms: float = 0.0):
        """
        Logs a specific internal event with FULL FIDELITY.
        """
        entry = {
            "type": event_type,
            "component": component,
            "duration_ms": round(duration_ms, 2), # Explicit latency tracking
            # We enforce a Deep Copy to capture the exact state at that millisecond
            # NO TRUNCATION.
            "data": copy.deepcopy(data) if isinstance(data, (dict, list)) else str(data)
        }
        self.run_data["trace_log"].append(entry)

    def save_report(self):
        filename = f"{self.current_run_id}_deep_trace.json"
        filepath = os.path.join(REPORTS_DIR, filename)
        with open(filepath, "w", encoding='utf-8') as f:
            json.dump(self.run_data, f, indent=2, ensure_ascii=False)
        print(f"💾 Deep Trace Report saved to: {filepath}")
        return filepath

recorder = DeepFlightRecorder()

In [2]:
# ==========================================
# CELL 2: INSTRUMENTED TOOLS (STOPWATCH EDITION)
# ==========================================

class TracedOmniRetriever(OmniRetriever):
    def retrieve(self, query, top_k_per_task=5, verbose=False):
        # 1. OPTIMIZATION TRACE
        t_start = time.perf_counter()
        omni = self.optimizer.optimize(query)
        duration = (time.perf_counter() - t_start) * 1000
        
        recorder.log_event("QUERY_DECOMPOSITION", "QueryOptimizer", omni, duration)
        
        final_structure = {"original_query": query, "tasks": []}
        
        # 2. ATOMIC EXECUTION TRACE
        for i, task in enumerate(omni['tasks']):
            sub_q = task['sub_query']
            candidates = {} 
            
            # --- A. VECTOR SEARCH (Full Text) ---
            t0 = time.perf_counter()
            vector_hits = []
            if task['hyde_passage']:
                emb = self.embedder.encode([task['hyde_passage']], convert_to_numpy=True)
                D, I = self.index.search(emb, k=top_k_per_task*2)
                for idx in I[0]: 
                    if idx < len(self.chunks): 
                        txt = self.chunks[idx]['text']
                        candidates[txt] = 0.0
                        vector_hits.append(txt) # FULL TEXT
            
            vec_dur = (time.perf_counter() - t0) * 1000
            recorder.log_event("SEARCH_VECTOR", f"Task_{i}", {
                "hyde_passage": task['hyde_passage'], # FULL TEXT
                "hit_count": len(vector_hits), 
                "full_results": vector_hits # FULL TEXT
            }, vec_dur)

            # --- B. GRAPH SEARCH (Full Text) ---
            t0 = time.perf_counter()
            graph_hits = []
            for entity in task['graph_entities']:
                if entity in self.graph_engine.G: 
                    facts = self.graph_engine.get_neighbors(entity)
                    for f in facts: 
                        candidates[f] = 0.0
                        graph_hits.append(f)
            
            graph_dur = (time.perf_counter() - t0) * 1000
            recorder.log_event("SEARCH_GRAPH", f"Task_{i}", {
                "entities": task['graph_entities'], 
                "hit_count": len(graph_hits), 
                "full_facts": graph_hits # FULL TEXT
            }, graph_dur)
            
            # --- C. RERANKING (Full Scores) ---
            t0 = time.perf_counter()
            unique_docs = list(candidates.keys())
            if not unique_docs:
                results = []
            else:
                pairs = [[sub_q, doc] for doc in unique_docs]
                scores = self.reranker.predict(pairs)
                ranked = sorted(list(zip(unique_docs, scores)), key=lambda x: x[1], reverse=True)
                results = ranked[:top_k_per_task]
                
                # LOG FULL SCORES
                score_log = [{"text": r[0], "score": float(r[1])} for r in results]
                
            rerank_dur = (time.perf_counter() - t0) * 1000
            recorder.log_event("RERANKER_SCORES", f"Task_{i}", score_log, rerank_dur)

            final_structure["tasks"].append({"sub_query": sub_q, "results": results})
            
        return final_structure

class TracedWebScout(DeepWebScout):
    def search_and_extract(self, sub_query: str):
        t0 = time.perf_counter()
        result = super().search_and_extract(sub_query)
        duration = (time.perf_counter() - t0) * 1000
        
        # Capture raw Tavily output structure
        recorder.log_event("WEB_SEARCH_RAW", "DeepWebScout", result, duration)
        return result

# INITIALIZE
print("⚙️  Injecting High-Fidelity Probes...")
traced_retriever = TracedOmniRetriever(GRAPH_PATH, CHUNKS_PATH, VECTOR_INDEX_PATH, BM25_INDEX_PATH, QUERY_MODEL)
traced_scout = TracedWebScout()
print("🚀 Probes Active.")

⚙️  Injecting High-Fidelity Probes...
📂 Loading Resources...
🚀 Omni-Retriever Ready.
🚀 Probes Active.


In [3]:
# ==========================================
# CELL 3: THE AGENT NODES
# ==========================================

class BrainState(TypedDict):
    query: str
    internal_knowledge: List[str]
    external_knowledge: List[str]
    gap_analysis: Dict
    final_answer: str

def retrieve_node(state: BrainState):
    query = state["query"]
    print(f"\n📚 [LIBRARIAN] Executing Traced Retrieval...")
    
    t0 = time.perf_counter()
    results = traced_retriever.retrieve(query, top_k_per_task=3)
    duration = (time.perf_counter() - t0) * 1000
    
    evidence = []
    for task in results.get("tasks", []):
        sub_q = task["sub_query"]
        for txt, score in task["results"]:
            evidence.append(f"[Score: {score:.2f}] {txt}")
    
    recorder.log_event("NODE_OUTPUT", "retrieve_node", {"evidence_count": len(evidence)}, duration)
    return {"internal_knowledge": evidence}

def audit_node(state: BrainState):
    print("🕵️‍♂️ [AUDITOR] Auditing Evidence & Freshness...")
    t0 = time.perf_counter()
    
    query = state["query"]
    evidence_text = "\n".join(state["internal_knowledge"][:20]) # Increased context limit
    
    sys_msg = """
        You are the Gap Analysis & Freshness Auditor.
        Your Job: Evaluate if the provided INTERNAL EVIDENCE is sufficient to answer the USER QUERY fully and accurately.

        ### CRITICAL "FRESHNESS" RULES:
        1. **Assume Stale Data:** Internal data is static. If the user asks for "current," "latest," "2024/2025," "today," or "news," and the evidence does not explicitly contain recent timestamps (last 30 days), you MUST mark it as **INSUFFICIENT**.
        2. **Dynamic Topics:** For queries about volatile topics (stock prices, weather, software versions, recent events), strictly reject internal data unless it is verified as live/real-time.
        3. **Trigger Search:** When rejecting data due to age/freshness, format your `missing_topics` specifically to guide a web search (e.g., use "Current status of X" or "2025 updates for Y").

        ### OUTPUT SCHEMA (Strict JSON):
        { 
            "sufficient": boolean, 
            "missing_topics": [
                "Topic 1 (e.g., 'Latest 2025 features for Python')",
                "Topic 2 (e.g., 'Current stock price of NVDA')"
            ] 
        }
        """
    user_msg = f"QUERY: {query}\n\nINTERNAL EVIDENCE:\n{evidence_text}"
    
    # Init LLM
    audit_llm = ChatGroq(temperature=0, model_name=AUDIT_MODEL, api_key=GROQ_API_KEY)
    
    response = audit_llm.invoke([
        SystemMessage(content=sys_msg),
        HumanMessage(content=user_msg)
    ])
    
    duration = (time.perf_counter() - t0) * 1000
    
    # LOG FULL PROMPT AND RESPONSE
    recorder.log_event("LLM_AUDIT", "AuditNode", {
        "full_system_prompt": sys_msg,
        "full_user_prompt": user_msg,
        "full_response": response.content
    }, duration)
    
    try:
        analysis = json.loads(response.content)
    except:
        analysis = {"sufficient": False, "missing_topics": [query]}
        
    if analysis.get("sufficient"):
        print("   ✅ Evidence is sufficient & fresh.")
    else:
        print(f"   ❌ Gaps/Stale Data detected: {analysis.get('missing_topics')}")

    return {"gap_analysis": analysis}

def web_search_node(state: BrainState):
    gaps = state["gap_analysis"].get("missing_topics", [])
    print(f"🌐 [SCOUT] Tracing Web Search for {len(gaps)} gaps...")
    external_facts = []
    
    t0 = time.perf_counter()
    for gap in gaps:
        res = traced_scout.search_and_extract(gap)
        if res["status"] == "success":
            external_facts.append(f"[Web: {gap}] {res.get('tavily_answer', '')}")
    duration = (time.perf_counter() - t0) * 1000
            
    recorder.log_event("NODE_OUTPUT", "web_search_node", {"facts_found": len(external_facts)}, duration)
    return {"external_knowledge": external_facts}

def synthesize_node(state: BrainState):
    print("✍️ [SYNTHESIZER] Writing Final Answer...")
    t0 = time.perf_counter()
    
    synth_llm = ChatGroq(temperature=0, model_name=SYNTHESIZE_MODEL, api_key=GROQ_API_KEY)
    
    sys_msg = """
        You are the **Chief Intelligence Officer**. 
        Your mandate is to synthesize fragmented information into a cohesive, executive-level intelligence briefing.

        ### CORE OBJECTIVES:
        1. **Executive Synthesis**: Do not just list facts. Synthesize them into a narrative that directly answers the user's intent. Start with a **Bottom Line Up Front (BLUF)** summary.
        2. **Hybrid Citation Protocol**: You must rigorously attribute every claim to its origin to maintain the chain of custody for information.
        - **Internal Data**: Cite as `[Internal Database]`.
        - **External Web Data**: Cite as `[Source: domain.com]`.
        - **Combined**: If a point is supported by both, use `[Internal Database | Source: domain.com]`.

        ### CONFLICT RESOLUTION:
        - If External and Internal sources conflict, present **both** viewpoints but prioritize the source with the more recent timestamp.
        - Explicitly label discrepancies: *"Note: Internal records indicate X, while recent public reporting suggests Y."*

        ### OUTPUT STRUCTURE:
        ## Executive Summary
        (A 2-3 sentence direct answer.)

        ## Detailed Analysis
        (Structured findings with 
        ## Strategic Implications / Next Steps
        (Actionable insights based on the data.)

        ### CONSTRAINT:
        - Answer ONLY using the provided context. If the context is missing specific details, state: "Insufficient intelligence available regarding [Topic]."
        """
    user_msg = f"""
    QUESTION: {state['query']}
    INTERNAL: {state['internal_knowledge']}
    EXTERNAL: {state.get('external_knowledge', [])}
    """
    
    response = synth_llm.invoke([
        SystemMessage(content=sys_msg),
        HumanMessage(content=user_msg)
    ])
    
    duration = (time.perf_counter() - t0) * 1000
    
    recorder.log_event("LLM_SYNTHESIS", "SynthesizeNode", {
        "full_system_prompt": sys_msg,
        "full_user_prompt": user_msg,
        "full_response": response.content
    }, duration)
    
    return {"final_answer": response.content}

In [5]:
# ==========================================
# CELL 4: EXECUTION
# ==========================================

workflow = StateGraph(BrainState)
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("audit", audit_node)
workflow.add_node("web_search", web_search_node)
workflow.add_node("synthesize", synthesize_node)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "audit")
workflow.add_conditional_edges("audit", 
    lambda x: "synthesize" if x["gap_analysis"].get("sufficient") else "web_search",
    {"synthesize": "synthesize", "web_search": "web_search"}
)
workflow.add_edge("web_search", "synthesize")
workflow.add_edge("synthesize", END)
app = workflow.compile()

from IPython.display import Markdown, display

def ask_brain_full_fidelity(question: str):
    recorder.start_run(question)
    
    print(f"\n❓ QUERY: {question}\n" + "="*40)
    
    try:
        result = app.invoke({"query": question})
        display(Markdown(result["final_answer"]))
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        path = recorder.save_report()
        print(f"\n📄 FULL FIDELITY LOG: {path}")

In [6]:
%%time
# TEST
ask_brain_full_fidelity("What is Beyonce's net worth and who is her husband?")

📼 Deep Trace started: trace_20260104_133034

❓ QUERY: What is Beyonce's net worth and who is her husband?

📚 [LIBRARIAN] Executing Traced Retrieval...
⚠️ Note: LLM returned flat format. Converting to Atomic Tasks...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/18 [00:00<?, ?it/s]

🕵️‍♂️ [AUDITOR] Auditing Evidence & Freshness...
   ❌ Gaps/Stale Data detected: ['Current net worth of Beyoncé (as of 2026)']
🌐 [SCOUT] Tracing Web Search for 1 gaps...
   🔎 Scouting External Cortex for: 'Current net worth of Beyoncé (as of 2026)'...
   👀 Retrieved 5 high-fidelity sources...
✍️ [SYNTHESIZER] Writing Final Answer...


## Executive Summary  
- **Bottom Line Up Front (BLUF):** Beyoncé’s net worth is reported at **$1.1 billion** in the most recent 2026 public sources, up from the **$780 million** figure cited by internal records as of June 2025. She is married to **Jay Z** (Shawn Carter).  

---

## Detailed Analysis  

### Net Worth  
| Source | Reported Net Worth | Date |
|--------|-------------------|------|
| Internal Database – Forbes estimate | **$780 million** | June 2025 [Internal Database] |
| External Web – Current reporting | **$1.1 billion** | 2026 [Source: web] |

**Discrepancy Note:** Internal records list Beyoncé’s net worth at $780 million (June 2025), while a recent external web source (2026) states she has reached billionaire status at $1.1 billion. The external figure is more recent and therefore takes precedence for current assessments, though the internal figure remains a valid historical benchmark.

### Marital Status  
- Beyoncé is **married to Jay Z (Shawn Carter)**. This relationship is documented in the internal database with a confidence score of 5.37 and corroborated by marriage date entries (April 4, 2008; public revelation October 22, 2008) [Internal Database].

No external source contradicts or updates this information.

---

## Strategic Implications / Next Steps  

1. **Financial Forecasting:**  
   - Adjust any valuation models or partnership considerations to reflect a **$1.1 billion** net worth as of 2026.  
   - Track upcoming Forbes or Bloomberg releases for confirmation and potential upward/downward revisions.

2. **Brand & Partnership Opportunities:**  
   - Beyoncé’s billionaire status enhances her leverage in brand negotiations, sponsorships, and joint ventures.  
   - Joint ventures with Jay Z should be evaluated as a **combined “billion‑dollar couple”** asset, given their historic joint earnings and market influence.

3. **Monitoring:**  
   - Set alerts for future **Forbes Celebrity 100** updates and any SEC filings related to Beyoncé’s business entities (e.g., Parkwood Entertainment, Ivy Park).  
   - Verify the external source’s credibility (domain, methodology) to ensure the $1.1 billion figure is robust before finalizing high‑stakes decisions.

💾 Deep Trace Report saved to: ./models/run_reports/trace_20260104_133034_deep_trace.json

📄 FULL FIDELITY LOG: ./models/run_reports/trace_20260104_133034_deep_trace.json
CPU times: user 491 ms, sys: 229 ms, total: 720 ms
Wall time: 16.9 s
